In [1]:
import pickle
fname = "s01.dat"
x = open('C:/Users/HP/Downloads/Deap Dataset/data_preprocessed_python/s01.dat','rb')
print(type(x))

<class '_io.BufferedReader'>


In [1]:
import pickle as pickle
with open('C:/Users/HP/Downloads/Deap Dataset/data_preprocessed_python/s01.dat', 'rb') as file:
    subject = pickle.load(file, encoding='latin1') 
    print(subject)

{'labels': array([[7.71, 7.6 , 6.9 , 7.83],
       [8.1 , 7.31, 7.28, 8.47],
       [8.58, 7.54, 9.  , 7.08],
       [4.94, 6.01, 6.12, 8.06],
       [6.96, 3.92, 7.19, 6.05],
       [8.27, 3.92, 7.  , 8.03],
       [7.44, 3.73, 7.08, 7.04],
       [7.32, 2.55, 6.32, 5.87],
       [4.04, 3.29, 3.62, 5.99],
       [1.99, 4.86, 2.04, 7.09],
       [2.99, 2.36, 3.63, 6.24],
       [2.71, 2.77, 3.4 , 7.35],
       [1.95, 3.12, 2.87, 6.18],
       [4.18, 2.24, 3.04, 5.04],
       [3.17, 8.08, 2.91, 5.04],
       [6.81, 7.44, 8.15, 7.14],
       [2.46, 6.91, 6.77, 6.41],
       [7.23, 7.15, 6.94, 8.01],
       [7.17, 8.  , 8.1 , 6.79],
       [8.26, 7.91, 7.19, 8.13],
       [9.  , 7.95, 8.37, 7.86],
       [7.09, 2.08, 7.06, 7.37],
       [8.15, 3.01, 7.37, 7.9 ],
       [7.04, 7.09, 8.01, 8.22],
       [8.86, 7.21, 8.65, 7.21],
       [7.28, 7.27, 7.41, 8.24],
       [7.35, 6.95, 7.03, 7.29],
       [3.88, 3.35, 4.01, 7.87],
       [1.36, 2.27, 3.  , 8.14],
       [2.08, 2.99, 3.22, 7.33],

In [2]:
!pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to c:\users\hp\appdata\local\temp\pip-req-build-k9h8dq3c
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=15951 sha256=239f6d0ecb8582ebc3b365ec6479b2ad7dff9201f491985a188a4004caf37a3f
  Stored in directory: C:\Users\HP\AppData\Local\Temp\pip-ephem-wheel-cache-2ymmz7jg\wheels\d6\50\90\4a327e4b939390a437827e4e0e2501da7207ea41967f53663a
Successfully built pyeeg


You should consider upgrading via the 'c:\users\hp\anaconda3\python.exe -m pip install --upgrade pip' command.


## Importing libraries

In [33]:
import numpy as np
import pickle as pickle
import pandas as pd
import pyeeg as pe

In [34]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#List of subjects

In [35]:
from sklearn.preprocessing import normalize

In [36]:

def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('C:/Users/HP/Downloads/Deap Dataset/data_preprocessed_python/s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolving the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        np.save('C:/Users/HP/Downloads/Deap Dataset/npyData/s' + sub, meta, allow_pickle=True, fix_imports=True)

In [37]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

In [38]:
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:
    with open('C:/Users/HP/Downloads/Deap Dataset/npyData/s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        print(sub.shape)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

np.save('C:/Users/HP/Downloads/Deap Dataset/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('C:/Users/HP/Downloads/Deap Dataset/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)


np.save('C:/Users/HP/Downloads/Deap Dataset/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('C:/Users/HP/Downloads/Deap Dataset/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

np.save('C:/Users/HP/Downloads/Deap Dataset/data_validation', np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save('C:/Users/HP/Downloads/Deap Dataset/label_validation', np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)


(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
(19520, 2)
training dataset: (468480, 70) (468480, 4)
testing dataset: (78080, 70) (78080, 4)
validation dataset: (78080, 70) (78080, 4)


### Loading Dataset and converting .npy to .csv

In [39]:
data= []
label = []
for subjects in subjectList:
     with open('C:/Users/HP/Downloads/Deap Dataset/npyData/s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
            data.append(sub[i][0])
            label.append(sub[i][1])
np.save('data', np.array(data), allow_pickle=True, fix_imports=True)
np.save('label', np.array(label), allow_pickle=True, fix_imports=True)

In [40]:
df=pd.DataFrame(data=data)
df.to_csv("data.csv",index=False)

df1=pd.DataFrame(data=label)
df1.to_csv("label.csv",index=False)

In [41]:
data1=pd.read_csv("C:/Users/HP/Downloads/Deap Dataset/data.csv")

In [42]:
data1

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,613.663621,914.264525,581.107620,1325.822606,1289.397695,839.213721,1020.221809,793.121658,1395.900243,1342.071643,...,1040.334395,981.162284,783.241786,1227.575104,1263.204863,990.496924,1092.856450,937.054171,1249.394169,1051.721009
1,630.957834,855.122300,542.239450,1315.474230,1423.398476,814.348323,981.989919,691.537720,1299.173642,1397.736807,...,1067.913800,1035.706019,751.708067,1163.560098,1275.857565,960.486529,1096.329289,864.969378,1266.382407,1116.827476
2,669.998695,931.945086,599.149975,1314.559510,1264.353974,784.051992,937.734317,772.203595,1229.182154,1299.614647,...,1011.983315,905.245619,850.154988,1199.205576,1309.021453,1059.582178,1114.917138,972.796731,1298.654550,1047.062139
3,701.677561,959.023161,504.419961,1297.271930,1157.961176,726.590780,960.032416,612.783460,1211.140861,1244.118359,...,950.509981,815.561911,728.121910,1204.177302,1297.992213,1011.433771,1143.999877,816.408425,1282.611507,940.885261
4,652.078144,902.875458,499.707842,1321.872766,1206.565845,711.020676,889.252456,589.541464,1308.288598,1357.399305,...,901.191224,747.678356,714.336361,1205.249989,1256.259389,795.775421,1053.693925,794.931176,1267.562506,1080.441384
5,653.101359,819.623927,663.554431,1421.859713,1154.308815,729.864673,813.499174,727.628134,1408.060486,1211.149772,...,823.773627,745.124969,701.917767,1162.142693,1229.957421,823.286455,928.100020,964.960062,1250.191396,958.960755
6,626.114235,791.979738,653.850325,1372.709734,1156.496350,730.786629,797.200730,733.916096,1406.331653,1235.489949,...,809.576199,909.449285,714.553942,1192.174755,1279.933695,848.102387,937.633501,988.078836,1226.899217,965.656511
7,634.585212,789.265455,668.735670,1374.168291,1110.809972,699.390568,807.251307,726.853786,1427.422027,1163.424971,...,828.511213,1149.307998,753.348744,1197.180666,1266.445314,862.682972,952.518359,972.314319,1195.717696,1047.581924
8,648.024695,739.041610,686.921380,1353.817883,1132.311875,700.470752,756.648796,779.937963,1434.341484,1200.879415,...,799.021025,1202.256479,759.597512,1204.812262,1284.597551,807.432556,977.414236,1043.103148,1199.599623,1101.229300
9,767.091981,1022.047591,721.905170,1371.118459,1122.096289,772.597391,966.262550,797.805901,1453.622624,1219.633697,...,807.634826,1174.663985,743.040299,1207.724421,1327.627741,884.205599,1046.569154,1148.072018,1196.176706,1095.561650


In [43]:
label1=pd.read_csv("C:/Users/HP/Downloads/Deap Dataset/label.csv")
label1

,0,1,2,3
0,7.71,7.60,6.9,7.83
1,7.71,7.60,6.9,7.83
2,7.71,7.60,6.9,7.83
3,7.71,7.60,6.9,7.83
4,7.71,7.60,6.9,7.83
5,7.71,7.60,6.9,7.83
6,7.71,7.60,6.9,7.83
7,7.71,7.60,6.9,7.83
8,7.71,7.60,6.9,7.83
9,7.71,7.60,6.9,7.83


In [44]:
x=data1.values

In [45]:
label1.isnull().sum()

0    0
1    0
2    0
3    0
dtype: int64

In [46]:
y_val=label1.loc[:,'0']

In [47]:
y_val.unique()

array([7.71, 8.1 , 8.58, 4.94, 6.96, 8.27, 7.44, 7.32, 4.04, 1.99, 2.99,
       2.71, 1.95, 4.18, 3.17, 6.81, 2.46, 7.23, 7.17, 8.26, 9.  , 7.09,
       8.15, 7.04, 8.86, 7.28, 7.35, 3.88, 1.36, 2.08, 3.03, 2.28, 3.81,
       2.06, 2.9 , 2.31, 3.33, 3.24, 5.1 , 8.01, 6.05, 5.04, 5.  , 4.96,
       4.99, 7.08, 8.94, 6.  , 8.97, 1.  , 4.06, 2.01, 4.87, 5.33, 7.21,
       7.55, 4.69, 6.92, 6.79, 5.45, 7.14, 7.33, 4.45, 7.94, 6.36, 7.91,
       7.29, 7.36, 7.15, 4.56, 7.1 , 8.14, 6.26, 3.65, 3.31, 3.45, 4.67,
       6.72, 5.59, 4.47, 3.87, 4.44, 5.17, 4.05, 4.72, 3.18, 5.01, 4.92,
       4.53, 4.33, 5.38, 6.68, 8.95, 6.1 , 2.94, 5.97, 3.15, 7.24, 4.12,
       2.51, 7.01, 4.9 , 8.17, 1.77, 1.86, 7.  , 6.14, 2.17, 1.85, 3.1 ,
       1.01, 2.03, 2.72, 1.79, 1.92, 6.97, 4.95, 8.24, 3.04, 8.09, 5.12,
       6.94, 5.96, 6.88, 8.05, 8.03, 4.  , 6.99, 7.96, 3.9 , 6.9 , 1.94,
       6.03, 4.86, 4.91, 1.91, 8.19, 6.83, 7.69, 7.05, 7.87, 7.37, 7.79,
       6.32, 5.06, 7.59, 7.41, 6.28, 7.56, 3.47, 3.

In [48]:
y_val

0         7.71
1         7.71
2         7.71
3         7.71
4         7.71
5         7.71
6         7.71
7         7.71
8         7.71
9         7.71
10        7.71
11        7.71
12        7.71
13        7.71
14        7.71
15        7.71
16        7.71
17        7.71
18        7.71
19        7.71
20        7.71
21        7.71
22        7.71
23        7.71
24        7.71
25        7.71
26        7.71
27        7.71
28        7.71
29        7.71
          ... 
624610    7.15
624611    7.15
624612    7.15
624613    7.15
624614    7.15
624615    7.15
624616    7.15
624617    7.15
624618    7.15
624619    7.15
624620    7.15
624621    7.15
624622    7.15
624623    7.15
624624    7.15
624625    7.15
624626    7.15
624627    7.15
624628    7.15
624629    7.15
624630    7.15
624631    7.15
624632    7.15
624633    7.15
624634    7.15
624635    7.15
624636    7.15
624637    7.15
624638    7.15
624639    7.15
Name: 0, Length: 624640, dtype: float64

In [49]:
y_val.values

array([7.71, 7.71, 7.71, ..., 7.15, 7.15, 7.15])

### Training and testing dataset

In [50]:
with open('data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
    
with open('label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)
    
X = normalize(X)
Z = np.ravel(Y[:, [1]])

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Domain_Train = np.ravel(Y[:, [2]])
Like_Train = np.ravel(Y[:, [3]])



with open('data_validation.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)
    
with open('label_validation.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [1]])

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Domain_Test = np.ravel(N[:, [2]])
Like_Test = np.ravel(N[:, [3]])

In [51]:
def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

## Random forest Regressor

In [56]:
# We have use differnet n_estimators like 10,100,250,500,750 
# But after 500 accuracy start decreasing so we stopped at 500.

In [ ]:
n_estimators = 10,  l1 error: 1.137919672131145 classification accuracy: 0.7774590163934426 1897 2440
n_estimators = 100, l1 error: 1.1029040163934432 classification accuracy: 0.8147540983606557 1988 2440
n_estimators = 100, l1 error: 1.274458098574928 classification accuracy: 0.7622950819672131 1860 2440
n_estimators = 100, l1 error: 1.4575897309409926 classification accuracy: 0.6823770491803278 1665 2440
n_estimators = 250, l1 error: 1.0905590819672137 classification accuracy: 0.830327868852459 2026 2440
n_estimators = 512, l1 error: 1.092375304175206 classification accuracy: 0.8364754098360656 2041 2440
n_estimators = 750, l1 error: 1.0953162021857932 classification accuracy: 0.8340163934426229 2035 2440
n_estimators = 750, l1 error: 1.066982950819674 classification accuracy: 0.8217213114754098 2005 2440

In [52]:
Aro_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Aro_R.fit(X[0:468480:32], Arousal_Train[0:468480:32])
testing (M, Arousal_Test, Aro_R)

l2 error: 2.0853847877452383 classification accuracy: 0.8229508196721311 2008 2440


In [53]:
Dom_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Dom_R.fit(X[0:468480:32], Domain_Train[0:468480:32])
testing (M, Domain_Test, Dom_R)

l2 error: 1.807351156865257 classification accuracy: 0.8237704918032787 2010 2440


In [54]:
Val_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Val_R.fit(X[0:468480:32], Valence_Train[0:468480:32])
testing (M, Valence_Test, Val_R)

l2 error: 1.8717262548223137 classification accuracy: 0.8348360655737705 2037 2440


In [60]:
from sklearn.ensemble import RandomForestRegressor
Lik_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Lik_R.fit(X[0:468480:32], Like_Train[0:468480:32])
testing (M, Like_Test, Lik_R)

l2 error: 2.4680703348667903 classification accuracy: 0.8540983606557377 2084 2440


## SVM

In [58]:
from sklearn import svm
clf = svm.SVR()
clf.fit(X[0:468480:32], Z[0:468480:32])

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [61]:
testing(M,Valence_Test,clf)

l2 error: 3.9956828328575873 classification accuracy: 0.6057377049180328 1478 2440


## Random forest classifier 

In [ ]:
#showing error bcoz our lables are continous 
#We are passing floats to a classifier which expects categorical values as the target vector.
# We can convert it to int but it will be like loss of information.
#Thats why using regressor algorithms.

In [29]:
#X = data1
#y = label1

In [30]:
# Splitting the data into training set and test set
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [32]:
# Fitting the classifier to the Training set
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state = 0)
#model.fit(X_train, y_train)

In [ ]:
# Predicting the test set result
#y_pred = model.predict(X_test)

In [ ]:
# Making the confusion matrix
#from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(y_test, y_pred)
#cm

In [59]:
# Getting the score for our model
#model.score(X_test, y_test)

## Adaboost Regressor

In [64]:
from sklearn.ensemble import AdaBoostRegressor
clf = AdaBoostRegressor(n_estimators=5000, learning_rate=0.01)
clf.fit(X[0:468480:32], Z[0:468480:32])
testing(M,Valence_Test,clf)

l2 error: 3.845887736564734 classification accuracy: 0.6217213114754099 1517 2440
